# Imports

In [ ]:

import utils
import data_utils 
import model_utils 

# reload the module
import importlib
importlib.reload(utils)
importlib.reload(data_utils)
importlib.reload(model_utils)


from utils import * 
from data_utils import *
from model_utils import *

: 

# Read data

In [101]:
limit_len = 100_000
data, labels = read_data(data_dir, original_labels = original_labels, original_features = original_features, seq_len = seq_len, offset = offset, limit_len = limit_len) 

train_data, pub_data, test_data = split_data_KD(data, labels) 
cat_labels = to_categorical(train_data[1], num_classes = len(original_labels))
cat_test_labels = to_categorical(test_data[1], num_classes = len(original_labels))

Number of files:  7
file:../HARB4/person 1/study01.txt has 51495 samples
file:../HARB4/person 1/study01.txt has 51495 samples
file:../HARB4/person 1/study00.txt has 85926 samples
file:../HARB4/person 1/study00.txt has 85926 samples
file:../HARB4/person 1/sleep00.txt has 520005 samples
file:../HARB4/person 1/sleep00.txt has 100000 samples
file:../HARB4/person 1/walk01.txt has 51150 samples
file:../HARB4/person 1/walk01.txt has 51150 samples
file:../HARB4/person 1/walk00.txt has 64185 samples
file:../HARB4/person 1/walk00.txt has 64185 samples
file:../HARB4/person 1/idle01.txt has 74466 samples
file:../HARB4/person 1/idle01.txt has 74466 samples
file:../HARB4/person 1/idle00.txt has 51120 samples
file:../HARB4/person 1/idle00.txt has 51120 samples


In [105]:
train_HARDataset = HARDataset(train_data[0], cat_labels)
test_HARDataset = HARDataset(test_data[0], cat_test_labels)
train_dataloader = DataLoader(train_HARDataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_HARDataset, batch_size=64, shuffle=True)



In [134]:


model = HARNet(n_lstm_layers = 2, n_features = 4, n_hidden = 30, n_classes = 4)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=lr)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [126]:
for i in range(5) : 
    
    train(model, criterion, optimizer, train_dataloader, device = device)

100%|██████████| 51/51 [02:28<00:00,  2.91s/it]


Train Accuracy: 0.294636Train Loss: 1.382035


100%|██████████| 51/51 [02:36<00:00,  3.06s/it]


Train Accuracy: 0.517887Train Loss: 1.348567


100%|██████████| 51/51 [02:27<00:00,  2.89s/it]


Train Accuracy: 0.569405Train Loss: 1.280947


100%|██████████| 51/51 [02:28<00:00,  2.92s/it]


Train Accuracy: 0.753676Train Loss: 1.219846


100%|██████████| 51/51 [02:28<00:00,  2.92s/it]

Train Accuracy: 0.755420Train Loss: 1.168679


In [127]:
for i in range(5) : 
    
    train(model, criterion, optimizer, train_dataloader, device = device)

100%|██████████| 51/51 [02:32<00:00,  2.99s/it]


Train Accuracy: 0.943627Train Loss: 1.120806


100%|██████████| 51/51 [02:30<00:00,  2.94s/it]


Train Accuracy: 0.967525Train Loss: 1.075161


100%|██████████| 51/51 [02:27<00:00,  2.90s/it]


Train Accuracy: 0.971201Train Loss: 1.032490


100%|██████████| 51/51 [02:26<00:00,  2.88s/it]


Train Accuracy: 0.973888Train Loss: 0.988918


100%|██████████| 51/51 [02:25<00:00,  2.85s/it]

Train Accuracy: 0.974736Train Loss: 0.946258


In [129]:
test(model, test_dataloader, privacy_engine=None, device = device)

100%|██████████| 15/15 [00:13<00:00,  1.09it/s]

Test Accuracy: 0.985539


# DP Training

## Notes on DP

In addition to all the considerations you normally take into account when choosing batch size, training models with DP adds another one - privacy cost.

Because of the threat model we assume and the way we add noise to the gradients, larger batch sizes (to a certain extent) generally help convergence. We add the same amount of noise to each gradient update (scaled to the norm of one sample in the batch) regardless of the batch size. What this means is that as the batch size increases, the relative amount of noise added decreases. while preserving the same epsilon guarantee.

You should, however, keep in mind that increasing batch size has its price in terms of epsilon, which grows at O(sqrt(batch_size)) as we train (therefore larger batches make it grow faster). The good strategy here is to experiment with multiple combinations of batch_size and noise_multiplier to find the one that provides the best possible quality at acceptable privacy guarantee.

There's another side to this - memory. Opacus computes and stores per sample gradients, so for every normal gradient, Opacus will store n=batch_size per-sample gradients on each step, thus increasing the memory footprint by at least O(batch_size). In reality, however, the peak memory requirement is O(batch_size^2) compared to a non-private model. This is because some intermediate steps in per sample gradient computation involve operations on two matrices, each with batch_size as one of the dimensions.

The good news is, we can pick the most appropriate batch size, regardless of memory constraints. Opacus has built-in support for virtual batches. Using it we can separate physical steps (gradient computation) and logical steps (noise addition and parameter updates): use larger batches for training, while keeping memory footprint low. Below we will specify two constants:

MAX_PHYSICAL_BATCH_SIZE defines the maximum batch size we can afford from a memory standpoint, and only affects computation speed
BATCH_SIZE, on the other hand, will affect only convergence and privacy guarantee.

___________________________________________________________________________________
noise_multiplier. It defines the trade-off between privacy and accuracy. Adding more noise will provide stronger privacy guarantees, but will also hurt model quality. In this run, the PrivacyEngine will determine this value based on the target values of EPSILON, DELTA, and EPOCHS. For the default settings, this will set noise_multiplier to about 0.4.
___________________________________________________________________________________
max_grad_norm. Defines the maximum magnitude of L2 norms to which we clip per sample gradients. There is a bit of tug of war with this threshold: on the one hand, a low threshold means that we will clip many gradients, hurting convergence, so we might be tempted to raise it. However, recall that we add noise with std=noise_multiplier * max_grad_norm so we will pay for the increased threshold with more noise. In most cases you can rely on the model being quite resilient to clipping (after the first few iterations your model will tend to adjust so that its gradients stay below the clipping threshold), so you can often just keep the default value (=1.0) and focus on tuning batch_size and noise_multiplier instead. That being said, sometimes clipping hurts the model so it may be worth experimenting with different clipping thresholds, like we are doing in this tutorial.
___________________________________________________________________________________


In [135]:
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(model, strict=False)
errors[-5:]

# model = ModuleValidator.fix(model)
# ModuleValidator.validate(model, strict=False)

[]

In [139]:

from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5
EPOCHS = 20

LR = 1e-3

model = HARNet(n_lstm_layers = 2, n_features = 4, n_hidden = 30, n_classes = 4)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=LR)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

ImportError: dlopen(/Users/gadmohamed/miniforge3/envs/fl/lib/python3.9/site-packages/functorch/_C.cpython-39-darwin.so, 0x0002): Symbol not found: (__ZN2at4_ops10as_strided4callERKNS_6TensorEN3c108ArrayRefINS5_6SymIntEEES8_NS5_8optionalIS7_EE)
  Referenced from: '/Users/gadmohamed/miniforge3/envs/fl/lib/python3.9/site-packages/functorch/_C.cpython-39-darwin.so'
  Expected in: '/Users/gadmohamed/miniforge3/envs/fl/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'